#### Copy baseline recognition statistics from firestore to BigQuery

In [4]:
import datetime

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db

cred = credentials.Certificate("./keys/avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275.json")
firebase_admin.initialize_app(cred)

In [2]:
#firestore
db = firestore.client()

In [33]:
#bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

project_id = 'biblosphere-210106'
credentials = service_account.Credentials.from_service_account_file('./keys/biblosphere-210106-46dbcd9058f7.json')

bq_client = bigquery.Client(credentials= credentials,project=project_id)

In [68]:
#create table
from google.cloud import bigquery

#client = bigquery.Client()

table_id = "biblosphere-210106.biblosphere.recognition_stats"

schema = [
    bigquery.SchemaField("timestamp", "NUMERIC", mode="REQUIRED"),
    bigquery.SchemaField("date", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("photo_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("algorithm", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("known_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("duration_microsec", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("recognized_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("total_finded_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("detectron_finded_books", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("photo_url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("added", "DATETIME", mode="REQUIRED"),
]

table = bigquery.Table(table_id, schema=schema)
table = bq_client.create_table(table)  
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table biblosphere-210106.biblosphere.recognition_stats


In [70]:
def insert_photo_to_bigquery(doc, table_id):
    data = doc.to_dict()
    
    today_date = datetime.datetime(2021, 9, 10)
    today_timestamp = today_date.timestamp()
    algoritm = "Denis's baseline algorithm (not ml)"
    known_books = None
    
    record = [{u'timestamp': today_timestamp,
              u'date': today_date.isoformat(),
              u'photo_id': doc.id,
              u'algorithm': algoritm,
              u'known_books': known_books,
              u'duration_microsec': None,
              u'recognized_books': data['recognized'] if 'recognized' in data else 0,
              u'total_finded_books': data['total'] if 'total' in data else 0,
              u'detectron_finded_books': None,
              u'photo_url': data['thumbnail'] if 'thumbnail' in data else None,
              u'added': datetime.datetime.today().isoformat(),
    }]
    #print(record)
    #print(data['thumbnail'])
    #print()
    
    errors = bq_client.insert_rows_json(table_id, record)  # Make an API request.
    if errors != []:
        print("Encountered errors while inserting rows (photo_id: {}): {}".format(doc.id, errors))
    
    

In [71]:
#insert from firestore

table_id = "biblosphere-210106.biblosphere.recognition_stats"    
docs = db.collection('photos').stream()
examples = 9999

count = 0
for doc in docs:
    if count == examples:
        break
    insert_photo_to_bigquery(doc, table_id)
    count += 1
    #print('inserted:', count, doc.id)
    
print('inserted:', count)

inserted: 1275


In [65]:
def photo_in_bigquery(photo_id):
    query = f"SELECT photo_id FROM `biblosphere-210106.biblosphere.recognition_stats` where photo_id = '{photo_id}' "    
    query_job = bq_client.query(query)
    results = list(query_job.result())
    return len(results) != 0

In [73]:
# inserted missed rows

table_id = "biblosphere-210106.biblosphere.recognition_stats"    
docs = db.collection('photos').stream()
examples = 9999

count = 0
for doc in docs:
    if count == examples:
        break
    if not photo_in_bigquery(doc.id):
        insert_photo_to_bigquery(doc, table_id)
    
    count += 1
    if count % 10 == 0:
        print(count)
    #print('inserted:', count, doc.id)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270


In [7]:
d = datetime.datetime(2021, 9, 10)
ts = d.timestamp()

d, ts

(datetime.datetime(2021, 9, 10, 0, 0), 1631232000.0)

In [20]:
datetime.datetime.today()

datetime.datetime(2021, 9, 10, 7, 10, 29, 213960)